In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

Mounted at /content/drive
Google Drive mounted successfully!


In [2]:
# ================================================================
# HIP DRR Trainer + Evaluator (IP-neutral) — Colab-ready program
# - Mounts Drive (optional), reads your existing hip_drr_128 dataset
# - Trains a compact tri-plane radiance field (MedNeRF-like core)
# - Evaluates in 3 sample runs and prints metrics
#   NON-DIAGNOSTIC. For education/planning demos only.
# ================================================================

from __future__ import annotations
import os, sys, json, math, time, random
from pathlib import Path
import subprocess

# -------------------------
# Install any missing deps
# -------------------------
def _ensure(pkgs):
    import importlib
    miss = []
    for p in pkgs:
        try:
            importlib.import_module(p)
        except Exception:
            miss.append(p)
    if miss:
        print("[setup] Installing:", miss)
        # Removed stdout=sys.stdout, stderr=sys.stderr
        subprocess.check_call([sys.executable, "-m", "pip", "install", *miss])

_ensure(["numpy", "Pillow", "tqdm", "imageio"])
import numpy as np
from PIL import Image
from tqdm import tqdm
import imageio

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Faster math on Ampere/Lovelace/Hopper
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")  # Windows OpenMP safety, harmless on Colab

# ================================================================
# CONFIG — EDIT THESE (point to your Drive)
# ================================================================
USE_DRIVE = True  # set False if not using Drive
DRIVE_DATA_ROOT = "/content/drive/MyDrive/SegmentedData"  # <-- EDIT
SPLITS_JSON     = "/content/drive/MyDrive/SegmentedData/splits.json"  # <-- EDIT (will be created if missing)
CKPT_PATH       = "/content/drive/MyDrive/SegmentedData/hip_tri_plane.pt"  # <-- EDIT

# Training hyperparameters (safe defaults across T4/L4/A100)
EPOCHS       = 6            # adjust up if you have time
BATCH_VIEWS  = 4            # increase on L4/A100 (e.g., 6–8)
PLANES_CH    = 24           # increase on L4/A100 (e.g., 32)
STEPS        = 96           # volume ray-march steps (96–128 typical)
LR           = 1e-3
SSIM_W       = 0.2
TV_W         = 1e-6
LATENT_REG_W = 1e-6
NUM_WORKERS  = 2
IMAGE_SIZE   = 128
N_VIEWS      = 72
STEP_DEG     = 5.0
AMP_DEFAULT  = True         # leave True for Colab GPUs
TIME_BUDGET_MIN = 0.0       # optional wall-clock stop (0 = ignore)

# ================================================================
# (Optional) Mount Drive
# ================================================================
if USE_DRIVE:
    try:
        from google.colab import drive  # type: ignore
        drive.mount('/content/drive')
    except Exception:
        print("[info] Not in Colab or Drive not available; continuing without mount.")

DATA_ROOT = Path(DRIVE_DATA_ROOT)
SPLITS    = Path(SPLITS_JSON)
CKPT      = Path(CKPT_PATH)
CKPT.parent.mkdir(parents=True, exist_ok=True)

# ================================================================
# Utilities: seed, metrics, dataset & geometry
# ================================================================
def set_seed(seed: int = 57000):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def psnr_from_mse(mse: torch.Tensor, max_val: float = 1.0) -> torch.Tensor:
    mse = torch.clamp(mse, 1e-12)
    return 10.0 * torch.log10((max_val ** 2) / mse)

def ssim_torch(x: torch.Tensor, y: torch.Tensor, C1=0.01**2, C2=0.03**2) -> torch.Tensor:
    w = torch.ones((1,1,7,7), device=x.device, dtype=x.dtype) / 49.0
    mu_x = F.conv2d(x, w, padding=3); mu_y = F.conv2d(y, w, padding=3)
    mu_x2 = mu_x*mu_x; mu_y2 = mu_y*mu_y; mu_xy = mu_x*mu_y
    sig_x2 = F.conv2d(x*x, w, padding=3) - mu_x2
    sig_y2 = F.conv2d(y*y, w, padding=3) - mu_y2
    sig_xy = F.conv2d(x*y, w, padding=3) - mu_xy
    ssim_n = (2*mu_xy + C1) * (2*sig_xy + C2)
    ssim_d = (mu_x2 + mu_y2 + C1) * (sig_x2 + sig_y2 + C2)
    return (ssim_n / (ssim_d + 1e-12)).mean()

def build_splits_if_missing(data_root: Path, splits_path: Path, val=0.15, test=0.15):
    if splits_path.exists():
        print("[splits] Using existing file:", splits_path)
        return
    series = sorted([d.name for d in data_root.iterdir() if d.is_dir() and (d/"drr_000.png").exists()])
    if not series:
        raise SystemExit(f"[error] No series folders with drr_000.png found under {data_root}")
    random.seed(57000)
    random.shuffle(series)
    n = len(series); nv = max(1, int(val*n)); nt = max(1, int(test*n))
    splits = {
        "train": series[:max(1, n-nv-nt)],
        "val":   series[max(1, n-nv-nt):max(1, n-nt)],
        "test":  series[max(1, n-nt):]
    }
    splits_path.write_text(json.dumps(splits, indent=2))
    print("[splits] Wrote new splits:", splits_path, "counts =", {k: len(v) for k, v in splits.items()})

# Parallel-beam rays for a turntable camera
def _ray_bundle(H: int, W: int, angle_deg: float, device: torch.device):
    ys = torch.linspace(-1.0, 1.0, H, device=device)
    xs = torch.linspace(-1.0, 1.0, W, device=device)
    yy, xx = torch.meshgrid(ys, xs, indexing='ij')
    plane = torch.stack([xx, yy, torch.zeros_like(xx)], dim=-1)  # [H,W,3]
    origin = torch.tensor([0.0, 0.0, -2.0], device=device)
    th = math.radians(float(angle_deg)); c, s = math.cos(th), math.sin(th)
    R = torch.tensor([[c,0.0,s],[0.0,1.0,0.0],[-s,0.0,c]], device=device)
    o = origin @ R.T; p = plane.reshape(-1, 3) @ R.T
    dirs = F.normalize(p - o, dim=-1); origins = o.expand_as(p)
    return origins, dirs

class RayCache:
    def __init__(self, H=128, W=128, n_views=72, step_deg=5.0, device=torch.device('cpu')):
        self.H, self.W, self.device = H, W, device
        self.angles = [i*step_deg for i in range(n_views)]
        self.cache = {a: _ray_bundle(H, W, a, device) for a in self.angles}
    def get(self, a: float):
        step = self.angles[1] - self.angles[0] if len(self.angles) > 1 else 5.0
        k = int(round((a % 360.0)/step)) % len(self.angles)
        return self.cache[self.angles[k]]

# ================================================================
# Model: tri-plane attenuation field + marcher (Beer–Lambert)
# ================================================================
class TriPlaneField(nn.Module):
    def __init__(self, C: int = 24, Fw: int = 32, latent_dim: int = 64):
        super().__init__()
        self.xy = nn.Parameter(torch.zeros(1, C, 128, 128))
        self.xz = nn.Parameter(torch.zeros(1, C, 128, 128))
        self.yz = nn.Parameter(torch.zeros(1, C, 128, 128))
        self.head = nn.Sequential(nn.Linear(3*C + latent_dim, Fw), nn.SiLU(), nn.Linear(Fw, 1))
        nn.init.normal_(self.xy, std=0.02); nn.init.normal_(self.xz, std=0.02); nn.init.normal_(self.yz, std=0.02)

    def _gs(self, im: torch.Tensor, ij: torch.Tensor) -> torch.Tensor:
        # Force grid_sample in FP32 to avoid AMP edge-cases
        g = ij.view(1, -1, 1, 2).to(dtype=torch.float32)
        return F.grid_sample(im.to(dtype=torch.float32), g, align_corners=True).squeeze(0).squeeze(-1).T

    def tv_loss(self) -> torch.Tensor:
        def tv(x):
            return (x[:,:,1:,:]-x[:,:,:-1,:]).abs().mean() + (x[:,:,:,1:]-x[:,:,:,:-1]).abs().mean()
        return tv(self.xy) + tv(self.xz) + tv(self.yz)

    def forward(self, pts: torch.Tensor, latent: torch.Tensor) -> torch.Tensor:
        x, y, z = pts.unbind(-1)
        feats = torch.cat([
            self._gs(self.xy, torch.stack([x,y], dim=-1)),
            self._gs(self.xz, torch.stack([x,z], dim=-1)),
            self._gs(self.yz, torch.stack([y,z], dim=-1)),
        ], dim=-1)
        if latent.ndim == 1:
            latent = latent.view(1, -1).expand(feats.size(0), -1)
        out = F.softplus(self.head(torch.cat([feats, latent], dim=-1)), beta=10.0)
        return torch.nan_to_num(out, 0.0, 1e6, 0.0)

def march_and_project(mu_fn, origins: torch.Tensor, dirs: torch.Tensor, *, steps=96, t0=0.0, t1=4.0,
                      chunk=16384, amp=True, fp32_exp=True) -> torch.Tensor:
    N = origins.shape[0]; device = origins.device
    t_vals = torch.linspace(t0, t1, steps, device=device)
    I = torch.ones(N, device=device)
    for s in range(0, N, chunk):
        e = min(N, s+chunk); o = origins[s:e]; d = dirs[s:e]
        Ii = torch.ones(e-s, device=device)
        for k in range(steps):
            P = o + d * t_vals[k]
            with torch.amp.autocast('cuda', enabled=amp):
                mu = mu_fn(P).squeeze(-1)
            mu = torch.nan_to_num(mu, 0.0, 1e6, 0.0)
            if fp32_exp:
                Ii *= torch.exp((-(t_vals[1]-t_vals[0]) * mu).float()).to(Ii.dtype)
            else:
                Ii *= torch.exp(-(t_vals[1]-t_vals[0]) * mu)
        Ii = torch.nan_to_num(Ii, 1.0, 1.0, 0.0)
        I[s:e] = Ii
    return torch.nan_to_num((1.0 - I).clamp_(0.0, 1.0), 0.0, 1.0, 0.0)

# ================================================================
# Dataset
# ================================================================
class HipDRRDataset(Dataset):
    def __init__(self, root: Path, split_series: list[str]):
        self.root = Path(root)
        self.items = []
        self.sid_to_idx = {sid: i for i, sid in enumerate(sorted(split_series))}
        for sid in split_series:
            meta_p = self.root/sid/"metadata.json"
            if not meta_p.exists():
                raise SystemExit(f"[error] Missing {meta_p}")
            meta = json.loads(meta_p.read_text())
            angles = meta.get("angles_deg", [i*5.0 for i in range(72)])
            for k, a in enumerate(angles):
                drr_path = self.root/sid/f"drr_{k:03d}.png"
                if not drr_path.exists():
                    raise SystemExit(f"[error] Missing DRR file: {drr_path}")
                self.items.append((sid, self.sid_to_idx[sid], k, float(a)))
    def __len__(self): return len(self.items)
    def __getitem__(self, idx):
        sid, sidx, k, ang = self.items[idx]
        im = Image.open(self.root/sid/f"drr_{k:03d}.png").convert("L")
        gt = torch.from_numpy(np.asarray(im, np.float32)/255.0).unsqueeze(0)  # [1,128,128]
        return sid, sidx, k, ang, gt

# ================================================================
# Train + Eval
# ================================================================
def one_epoch(model, embed, loader, optim, device, amp, ssim_w, steps, H, W, chunk, raycache, tv_w, lat_reg_w):
    model.train()
    scaler = torch.amp.GradScaler('cuda', enabled=amp)
    tot_mse = tot_ssim = n = 0.0
    for _, sidx, _, ang, gt in loader:
        sidx = sidx.to(device); ang = ang.to(device); gt = gt.to(device)
        optim.zero_grad(set_to_none=True)
        preds = []
        with torch.amp.autocast('cuda', enabled=amp):
            for b in range(gt.size(0)):
                lat = embed(sidx[b])
                o, d = raycache.get(float(ang[b].item()))
                mu_fn = lambda P: model(P, lat)
                drr = march_and_project(mu_fn, o, d, steps=steps, amp=amp, fp32_exp=True).view(1,1,H,W)
                preds.append(drr)
            pred_b = torch.cat(preds, dim=0)
            if not torch.isfinite(pred_b).all():
                raise RuntimeError("Non-finite prediction; try AMP off or lower --steps.")
            mse  = F.mse_loss(pred_b, gt)
            ssim = ssim_torch(pred_b, gt)
            loss = mse + ssim_w*(1.0 - ssim) + tv_w*model.tv_loss() + lat_reg_w*embed.weight.pow(2).mean()
        scaler.scale(loss).backward(); scaler.step(optim); scaler.update()
        tot_mse += float(mse.detach().cpu())*gt.size(0)
        tot_ssim+= float(ssim.detach().cpu())*gt.size(0)
        n += gt.size(0)
    return tot_mse/max(1,n), tot_ssim/max(1,n)

@torch.no_grad()
def eval_loop(model, embed, loader, device, amp, steps, H, W, chunk, raycache):
    model.eval()
    tot_mse = tot_ssim = n = 0.0
    for _, sidx, _, ang, gt in loader:
        sidx=sidx.to(device); ang=ang.to(device); gt=gt.to(device)
        preds=[]
        with torch.amp.autocast('cuda', enabled=amp):
            for b in range(gt.size(0)):
                lat=embed(sidx[b])
                o, d = raycache.get(float(ang[b].item()))
                mu_fn = lambda P: model(P, lat)
                drr = march_and_project(mu_fn, o, d, steps=steps, amp=amp, fp32_exp=True).view(1,1,H,W)
                preds.append(drr)
            pred_b = torch.cat(preds, dim=0)
            mse  = F.mse_loss(pred_b, gt)
            ssim = ssim_torch(pred_b, gt)
        tot_mse += float(mse.detach().cpu())*gt.size(0)
        tot_ssim+= float(ssim.detach().cpu())*gt.size(0)
        n += gt.size(0)
    mse_mean = tot_mse/max(1,n)
    psnr_mean= float(psnr_from_mse(torch.tensor(mse_mean)).item())
    ssim_mean= tot_ssim/max(1,n)
    return {"mse": mse_mean, "psnr": psnr_mean, "ssim": ssim_mean}

def train_and_eval():
    set_seed(57000)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    amp = AMP_DEFAULT and (device.type == "cuda")
    print(f"[env] Using device: {device} | AMP: {amp}")
    if device.type == "cuda":
        print("      GPU:", torch.cuda.get_device_name(0))

    # Verify data root & splits
    if not DATA_ROOT.exists():
        raise SystemExit(f"[error] DATA_ROOT does not exist: {DATA_ROOT}")
    build_splits_if_missing(DATA_ROOT, SPLITS)
    splits = json.loads(SPLITS.read_text())
    print("[splits] counts:", {k: len(v) for k, v in splits.items()})

    H = W = IMAGE_SIZE
    train_ds = HipDRRDataset(DATA_ROOT, splits["train"])
    val_ds   = HipDRRDataset(DATA_ROOT, splits["val"])   if len(splits["val"])  else None
    test_ds  = HipDRRDataset(DATA_ROOT, splits["test"])  if len(splits["test"]) else None

    train_ld = DataLoader(train_ds, batch_size=BATCH_VIEWS, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda"))
    val_ld   = DataLoader(val_ds,   batch_size=BATCH_VIEWS, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda")) if val_ds else None
    test_ld  = DataLoader(test_ds,  batch_size=BATCH_VIEWS, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda")) if test_ds else None

    # Model & optimizer
    model = TriPlaneField(C=PLANES_CH, Fw=32, latent_dim=64).to(device)
    embed = nn.Embedding(num_embeddings=len(train_ds.sid_to_idx), embedding_dim=64).to(device)
    nn.init.normal_(embed.weight, std=0.02)
    optim = torch.optim.Adam(list(model.parameters()) + list(embed.parameters()), lr=LR)

    # Ray cache for the fixed turntable
    rc = RayCache(H, W, n_views=N_VIEWS, step_deg=STEP_DEG, device=device)

    # Train
    best_val = float("inf")
    run_start = time.perf_counter()
    for epoch in range(1, EPOCHS+1):
        t0 = time.perf_counter()
        tr_mse, tr_ssim = one_epoch(model, embed, train_ld, optim, device, amp, SSIM_W, STEPS, H, W, 16384, rc, TV_W, LATENT_REG_W)
        tr_psnr = float(psnr_from_mse(torch.tensor(tr_mse)).item())
        sec = time.perf_counter() - t0
        print(f"Epoch {epoch:03d} | {sec:.1f}s | train: MSE={tr_mse:.6f} PSNR={tr_psnr:.2f} SSIM={tr_ssim:.3f}")

        if val_ld is not None:
            vm = eval_loop(model, embed, val_ld, device, amp, STEPS, H, W, 16384, rc)
            print(f"             val: MSE={vm['mse']:.6f} PSNR={vm['psnr']:.2f} SSIM={vm['ssim']:.3f}")
            if vm["mse"] < best_val:
                best_val = vm["mse"]
                torch.save({"model": model.state_dict(), "embed": embed.state_dict(),
                            "optim": optim.state_dict(), "epoch": epoch}, CKPT)
                print("Saved best checkpoint ->", CKPT)

        if TIME_BUDGET_MIN > 0:
            elapsed_min = (time.perf_counter() - run_start)/60.0
            if elapsed_min >= TIME_BUDGET_MIN:
                print(f"[stop] Time budget reached ({elapsed_min:.1f} min).")
                break

    # Save last (rolling)
    last = CKPT.with_name(CKPT.stem + "_last.pt")
    torch.save({"model": model.state_dict(), "embed": embed.state_dict(),
                "optim": optim.state_dict(), "epoch": epoch}, last)
    print("Saved last checkpoint ->", last)

    # ============================================================
    # Three evaluation runs (prints metrics)
    # ============================================================
    @torch.no_grad()
    def load_for_eval(ckpt_path: Path):
        ck = torch.load(ckpt_path, map_location=device)
        m = TriPlaneField(C=PLANES_CH, Fw=32, latent_dim=64).to(device)
        e = nn.Embedding(num_embeddings=len(train_ds.sid_to_idx), embedding_dim=64).to(device)
        m.load_state_dict(ck["model"]); e.load_state_dict(ck["embed"])
        return m, e

    # Run 1: Full validation split, default STEPS
    if val_ld is not None:
        m1, e1 = load_for_eval(CKPT if CKPT.exists() else last)
        vm1 = eval_loop(m1, e1, val_ld, device, amp, STEPS, H, W, 16384, rc)
        print("\n[EVAL RUN 1] Split=val, steps=", STEPS, " -> ",
              f"MSE={vm1['mse']:.6f}  PSNR={vm1['psnr']:.2f}  SSIM={vm1['ssim']:.3f}")

    # Run 2: Full test split (if present), default STEPS
    if test_ld is not None:
        m2, e2 = load_for_eval(CKPT if CKPT.exists() else last)
        vm2 = eval_loop(m2, e2, test_ld, device, amp, STEPS, H, W, 16384, rc)
        print("[EVAL RUN 2] Split=test, steps=", STEPS, " -> ",
              f"MSE={vm2['mse']:.6f}  PSNR={vm2['psnr']:.2f}  SSIM={vm2['ssim']:.3f}")

    # Run 3: Validation split with higher marcher steps (if present), STEPS+32
    if val_ld is not None:
        m3, e3 = load_for_eval(CKPT if CKPT.exists() else last)
        steps_hi = STEPS + 32
        vm3 = eval_loop(m3, e3, val_ld, device, amp, steps_hi, H, W, 16384, rc)
        print("[EVAL RUN 3] Split=val, steps=", steps_hi, " -> ",
              f"MSE={vm3['mse']:.6f}  PSNR={vm3['psnr']:.2f}  SSIM={vm3['ssim']:.3f}")

    # Optional: export a 72‑frame GIF of the first series in val split
    SAVE_GIF = True
    if SAVE_GIF and val_ld is not None and len(splits["val"]) > 0:
        sid0 = splits["val"][0]
        frames = [Image.open(DATA_ROOT/sid0/f"drr_{i:03d}.png") for i in range(72)]
        gif_path = CKPT.parent / "turntable_val0.gif"
        imageio.mimsave(gif_path, frames, duration=0.08)
        print("Saved turntable GIF ->", gif_path)

# ================================================================
# Execute
# ================================================================
train_and_eval()

[setup] Installing: ['Pillow']
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[env] Using device: cuda | AMP: True
      GPU: NVIDIA A100-SXM4-40GB
[splits] Using existing file: /content/drive/MyDrive/SegmentedData/splits.json
[splits] counts: {'train': 20, 'val': 4, 'test': 4}
Epoch 001 | 602.9s | train: MSE=0.104578 PSNR=9.81 SSIM=0.283
             val: MSE=0.211681 PSNR=6.74 SSIM=0.245
Saved best checkpoint -> /content/drive/MyDrive/SegmentedData/hip_tri_plane.pt
Epoch 002 | 244.0s | train: MSE=0.200299 PSNR=6.98 SSIM=0.235
             val: MSE=0.211681 PSNR=6.74 SSIM=0.245
Epoch 003 | 243.1s | train: MSE=0.200299 PSNR=6.98 SSIM=0.235
             val: MSE=0.211681 PSNR=6.74 SSIM=0.245
Epoch 004 | 244.7s | train: MSE=0.200299 PSNR=6.98 SSIM=0.235
             val: MSE=0.211681 PSNR=6.74 SSIM=0.245
Epoch 005 | 244.1s | train: MSE=0.200299 PSNR=6.98 SSIM=0.235
             val: MSE=0.211681 PSNR=6.74 S

In [4]:
import os
from pathlib import Path
import shutil

DRIVE_DATA_ROOT = "/content/drive/MyDrive/SegmentedData" # Make sure this matches your configuration

data_root = Path(DRIVE_DATA_ROOT)

if not data_root.exists():
    print(f"Error: Data root not found at {data_root}")
else:
    for series_folder in data_root.iterdir():
        if series_folder.is_dir():
            # Look for a subfolder containing drr_000.png
            extra_folder_path = None
            for sub_item in series_folder.iterdir():
                if sub_item.is_dir() and (sub_item / "drr_000.png").exists():
                    extra_folder_path = sub_item
                    break # Assuming only one such folder per series

            if extra_folder_path:
                print(f"Found data in subfolder: {extra_folder_path.name}")
                print(f"Moving files from {extra_folder_path} to {series_folder}")
                for item in extra_folder_path.iterdir():
                    try:
                        shutil.move(str(item), str(series_folder))
                        print(f"Moved {item.name}")
                    except Exception as e:
                        print(f"Error moving {item.name}: {e}")
                # Optional: remove the now empty extra folder
                try:
                    os.rmdir(str(extra_folder_path))
                    print(f"Removed empty folder {extra_folder_path}")
                except OSError:
                    # Folder might not be empty if there were errors moving files
                    print(f"Could not remove {extra_folder_path}, it may not be empty.")
            else:
                print(f"No subfolder with 'drr_000.png' found in {series_folder}, skipping.")

print("File reorganization complete.")

Found data in subfolder: e64c0816358c
Moving files from /content/drive/MyDrive/SegmentedData/e64c0816358c/e64c0816358c to /content/drive/MyDrive/SegmentedData/e64c0816358c
Moved drr_000.png
Moved drr_001.png
Moved drr_002.png
Moved drr_003.png
Moved drr_004.png
Moved drr_005.png
Moved drr_006.png
Moved drr_007.png
Moved drr_008.png
Moved drr_009.png
Moved drr_010.png
Moved drr_011.png
Moved drr_012.png
Moved drr_013.png
Moved drr_014.png
Moved drr_015.png
Moved drr_016.png
Moved drr_017.png
Moved drr_018.png
Moved drr_019.png
Moved drr_020.png
Moved drr_021.png
Moved drr_022.png
Moved drr_023.png
Moved drr_024.png
Moved drr_025.png
Moved drr_026.png
Moved drr_027.png
Moved drr_028.png
Moved drr_029.png
Moved drr_030.png
Moved drr_031.png
Moved drr_032.png
Moved drr_033.png
Moved drr_034.png
Moved drr_035.png
Moved drr_036.png
Moved drr_037.png
Moved drr_038.png
Moved drr_039.png
Moved drr_040.png
Moved drr_041.png
Moved drr_042.png
Moved drr_043.png
Moved drr_044.png
Moved drr_045.png
